In [ ]:
from exod.processing.bayesian_computations import precompute_bayes_1000, load_precomputed_bayes1000, \
    load_precomputed_bayes_limits, precompute_bayes_limits, get_cube_masks_peak_and_eclipse, B_peak, B_eclipse, \
    PrecomputeBayesLimits
from exod.utils.plotting import cmap_image
from exod.utils.logger import logger
from exod.pre_processing.data_loader import DataLoader
from exod.utils.util import save_df, save_info, get_unique_xy
from exod.xmm.event_list import EventList
from exod.xmm.observation import Observation
from exod.processing.template_based_background_inference import compute_expected_cube_using_templates
from exod.processing.coordinates import get_regions_sky_position, calc_df_regions

from random import shuffle
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.colors import ListedColormap, BoundaryNorm
import pandas as pd
from astropy.visualization import ImageNormalize, SqrtStretch
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
from matplotlib import cm

In [ ]:
size_arcsec= 20.0
time_interval= 20
gti_threshold= 1.5
min_energy= 0.2
max_energy= 2.0
gti_only= False
remove_partial_ccd_frames= False
clobber= False

observation = Observation('0823432701')
observation.filter_events(clobber=clobber)
observation.create_images(clobber=clobber)
observation.get_files()
observation.get_events_overlapping_subsets()

for i_subset, subset_overlapping_exposures in enumerate(observation.events_overlapping_subsets):
    savedir = observation.path_results / f'subset_{i_subset}'
    savedir.mkdir(exist_ok=True)
    
    event_list = EventList.from_event_lists(subset_overlapping_exposures)
    dl = DataLoader(event_list=event_list, time_interval=time_interval, size_arcsec=size_arcsec,
                    gti_only=gti_only, min_energy=min_energy, max_energy=max_energy,
                    gti_threshold=gti_threshold, remove_partial_ccd_frames=remove_partial_ccd_frames)
    dl.run()

In [ ]:
ax = plt.figure(figsize=(10,10)).add_subplot(projection='3d')
for i in range(1000):
    im = dl.data_cube.data[:,:,i]
    #im = np.ma.masked_where(im == 0, data)
    data = im
    
    xx, yy = np.meshgrid(np.linspace(0, 1,im.shape[1]), np.linspace(0,1,im.shape[0]))
    X =  xx 
    Y =  yy
    Z =  10*np.ones(X.shape)
    cset = ax.contourf(data, X, Y, np.unique(im), zdir='x', offset=i/50, cmap=cmap_image(), levels=np.arange(1,10))
    
    #ax.set_zlim((0.,1.))
    ax.view_init(17,-150,0)
    
    #plt.colorbar(cset, shrink=0.5)
plt.show()
    
    


In [ ]:
plt.show()